In [ ]:
# Link Trabalho Final : https://www.dropbox.com/s/3fla2xuc8t9xba2/Deep%20Learning%20-%20Lista%20de%20Exerc%C3%ADcios%20v1.4.pdf?dl=0

In [ ]:
from sklearn import datasets
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/__FIAP/MBA - 17IA/Deep Learning & Reinforcement Learning/dados_trabalho_final/DadosSpotify.csv'

'/content/drive/MyDrive/__FIAP/MBA - 17IA/Deep Learning & Reinforcement Learning/dados_trabalho_final/DadosSpotify.csv'


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/__FIAP/MBA - 17IA/Deep Learning & Reinforcement Learning/dados_trabalho_final/DadosSpotify.csv')

print(df.shape)
df.head(5)

(2017, 17)


,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [ ]:
df.target.value_counts()

1    1020
0     997
Name: target, dtype: int64

In [ ]:
import keras
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, AveragePooling2D
from keras.utils import np_utils
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import  to_categorical

In [ ]:
# Separando dados em X e Y
X= df.drop(['target','song_title','artist','key','mode'],axis=1).to_numpy()
Y= df['target'].to_numpy()
print(X.shape, Y.shape, df.columns)

from sklearn.model_selection import train_test_split

X_TEMP,X_test,y_TEMP, y_test = train_test_split(X,Y,test_size = 0.2 , random_state = 42 )

(2017, 12) (2017,) Index(['id', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'target',
       'song_title', 'artist'],
      dtype='object')


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print("Shape X_train:",X_train.shape)
print("Shape X_test:",X_test.shape)
print("Shape y_train:",y_train.shape)
print("Shape y_test:",y_test.shape)
type(X_train)

Shape X_train: (1613, 12)
Shape X_test: (404, 12)
Shape y_train: (1613,)
Shape y_test: (404,)


numpy.ndarray

In [ ]:
modelo = Sequential()
modelo.add(Dense(activation = "relu", input_dim = 12, 
                     units = 8, kernel_initializer = "uniform"))
modelo.add(Dense(activation = "sigmoid", units = 1, 
                     kernel_initializer = "uniform"))
modelo.compile(optimizer = 'adam' , loss = 'binary_crossentropy', 
                   metrics = ['accuracy'] )
modelo.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelo.fit(X_train , y_train ,epochs = 10)

Epoch 1/10
51/51 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.7434
Epoch 2/10
51/51 [==============================] - 0s 1ms/step - loss: 0.6709 - accuracy: 0.8807
Epoch 3/10
51/51 [==============================] - 0s 1ms/step - loss: 0.6155 - accuracy: 0.8740
Epoch 4/10
51/51 [==============================] - 0s 1ms/step - loss: 0.5302 - accuracy: 0.8893
Epoch 5/10
51/51 [==============================] - 0s 1ms/step - loss: 0.4363 - accuracy: 0.9058
Epoch 6/10
51/51 [==============================] - 0s 2ms/step - loss: 0.3725 - accuracy: 0.9075
Epoch 7/10
51/51 [==============================] - 0s 1ms/step - loss: 0.3011 - accuracy: 0.9435
Epoch 8/10
51/51 [==============================] - 0s 1ms/step - loss: 0.2494 - accuracy: 0.9577
Epoch 9/10
51/51 [==============================] - 0s 1ms/step - loss: 0.2151 - accuracy: 0.9597
Epoch 10/10
51/51 [==============================] - 0s 1ms/step - loss: 0.1791 - accuracy: 0.9730


In [ ]:
scores = modelo.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (modelo.metrics_names[1], scores[1]*100))

13/13 [==============================] - 0s 1ms/step - loss: 0.1547 - accuracy: 0.9827

accuracy: 98.27%


In [ ]:
scores = modelo.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (modelo.metrics_names[1], scores[1]*100))

51/51 [==============================] - 0s 866us/step - loss: 0.1646 - accuracy: 0.9752

accuracy: 97.52%
